In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
open_api_key = os.environ["OPENAI_API_KEY"]

# Loading Completion Model

In [5]:
from langchain_openai import OpenAI
llm_model = OpenAI()

# Loading the Chat Completion Model

In [15]:
from langchain_openai import ChatOpenAI
chat_model = ChatOpenAI(model="gpt-4o-mini")

# Prompts and Prompt Template

In [7]:
## Prompt template for completion model
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "You are an expert Cricket Journalist, covering cricket since 1980s. Tell me a {adjective} story about {topic}"
)
llm_model_prompt = prompt_template.format(
    adjective= "fascinating",
    topic= "Sourav Ganguly"
)
response =llm_model.invoke(llm_model_prompt)
print(response)



One of the most fascinating stories about Sourav Ganguly dates back to the 2001 Border-Gavaskar Trophy, where India was facing Australia in a highly competitive series.

In the third test match at Eden Gardens, Australia had set a mammoth target of 384 runs for India to chase in the fourth innings. The Indian team, led by Ganguly, was under immense pressure and many believed that they had no chance of winning the match.

However, Ganguly had other plans. He walked out to bat with a determined mindset and played one of the most iconic innings in Indian cricket history. He scored a brilliant century, leading India to a historic victory and helping them level the series 1-1.

But what many people don't know is that Ganguly almost didn't play that match. He was suffering from a severe back injury and even had to take injections to reduce the pain. The team management had even considered dropping him from the playing XI, but Ganguly insisted on playing and assured them that he would give 

In [16]:
## Prompt template for the chat completion model

from langchain_core.prompts import  ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a {profession} expert on {topic}."),
        ("human", "Hello, Mr. {profession}, can you please answer a question"),
        ("ai", "Sure!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    profession = "Cricket Journalist",
    topic = "Sachin Tendulkar",
    user_input = "How many times Sachin Got out in 90s in his international Cricket Carrer?"
)

response = chat_model.invoke(messages)
print(response.content)

Sachin Tendulkar was dismissed in the 90s a total of 10 times during his international cricket career. This includes both One Day Internationals (ODIs) and Test matches. His ability to score runs consistently was remarkable, but falling short of the century on these occasions was a notable aspect of his career. If you have any more questions about Sachin Tendulkar or his career, feel free to ask!


# Few-Shot Prompting Template 

In [18]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
# Step 1: Create Examples
examples = [
    {"input": "hi!", "output": "hola!"},
    {"input": "bye!", "output": "adios!"},
]

# Step 2: Create an example Prompt
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Step 3: Create a few-shot prompt using examples and example prompt
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples= examples
)

# Step 4: Create Final Prompt
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are an English-Spanish translator."),
        few_shot_prompt,
        ("human","{input}"),
    ]
)

# Step 5: Create input message
messages = final_prompt.format_messages(
    input= "Hello, How are you doing? Hope you're having a great day."
)

# Step 6: Invoke the LLM
response = chat_model.invoke(messages)
print(response.content)



¡Hola! ¿Cómo estás? Espero que estés teniendo un gran día.


# Chains

In [19]:
# Running the above example using chain

from langchain_core.prompts import FewShotChatMessagePromptTemplate
# Step 1: Create Examples
examples = [
    {"input": "hi!", "output": "hola!"},
    {"input": "bye!", "output": "adios!"},
]

# Step 2: Create an example Prompt
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Step 3: Create a few-shot prompt using examples and example prompt
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples= examples
)

# Step 4: Create Final Prompt
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are an English-Spanish translator."),
        few_shot_prompt,
        ("human","{input}"),
    ]
)

# Step 5: Create chain
chain = final_prompt | chat_model


# Step 6: Invoke the chain
response = chain.invoke({"input": "Hello, How are you doing? Hope you're having a great day."})
print(response.content)


Hola, ¿cómo estás? Espero que estés teniendo un gran día.


# Output Parser

In [20]:
# Step 1 : Initialize the Chat Model

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [22]:
## Output parser for completion model
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser

## Step 1: Initialize model
llm_model = OpenAI()

## Step 2: Create Prompt Template
json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answe` key that answers the following question: {question}"
)

## Step 3: Initialize the Output Parser
json_parser = SimpleJsonOutputParser()

## Step 4: Create the Chain
json_chain = json_prompt|llm_model|json_parser

## Step 5: Invoke the chain with the input
response = json_chain.invoke({"question":"which is the most populated Country"})
print(response['answer'])


China


In [23]:
## Defining a Custom Output format using pydantic
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [24]:
# Step 1: Define a class

class Joke(BaseModel):
    setup: str =Field(description="Question to Setup a joke")
    punchline: str = Field(description="answer to resolve the Joke")

In [25]:
# Step 2: Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Step 3: Create the prompt with parser format instructions
prompt = PromptTemplate(
    template= "Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions":parser.get_format_instructions()},
)

# Step 4: Create a chain with the prompt amd the parser
chain = prompt|llm_model|parser

# Step 5: invoke the chain
chain.invoke({"query":"Tell me a Joke"})

{'setup': 'Why did the tomato turn red?',
 'punchline': 'Because it saw the salad dressing!'}